## 1. Import libraries and preprocess dataset

In [20]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

# Training dataset
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')

# Test dataset
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


## 2. Create the CNN model

In [24]:
model = tf.keras.models.Sequential()

# Convolution layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Pooling layer
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# 2nd Convolution layer
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Flattening layer
model.add(tf.keras.layers.Flatten())

# Full connection layer
model.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile with the descent algorithm and loss function
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## 3. Train the CNN model

In [26]:
model.fit(x=training_set, validation_data=test_set, epochs=25)

Epoch 1/25
250/250 [==============================] - 76s 300ms/step - loss: 0.6856 - accuracy: 0.5621 - val_loss: 0.6485 - val_accuracy: 0.6275
Epoch 2/25
250/250 [==============================] - 62s 249ms/step - loss: 0.6453 - accuracy: 0.6357 - val_loss: 0.6080 - val_accuracy: 0.6790
Epoch 3/25
250/250 [==============================] - 57s 226ms/step - loss: 0.6075 - accuracy: 0.6718 - val_loss: 0.6036 - val_accuracy: 0.6940
Epoch 4/25
250/250 [==============================] - 71s 285ms/step - loss: 0.5805 - accuracy: 0.6942 - val_loss: 0.5729 - val_accuracy: 0.7130
Epoch 5/25
250/250 [==============================] - 64s 255ms/step - loss: 0.5514 - accuracy: 0.7174 - val_loss: 0.5741 - val_accuracy: 0.7050
Epoch 6/25
250/250 [==============================] - 64s 256ms/step - loss: 0.5200 - accuracy: 0.7389 - val_loss: 0.5844 - val_accuracy: 0.7040
Epoch 7/25
250/250 [==============================] - 66s 263ms/step - loss: 0.5005 - accuracy: 0.7544 - val_loss: 0.5358 - val_ac

## 4. Predict the following observation

In [27]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices

print(f'Prediction: {"dog" if result[0][0] == 1 else "cat"}')

Prediction: dog
